In [3]:
import sympy as sp
from scipy.integrate import odeint
import numpy as np
import matplotlib.pyplot as plt

In [4]:
# Symbols for time and generalized coordinates
t = sp.symbols('t')
theta1, theta2 = sp.Function('theta1')(t), sp.Function('theta2')(t)
L1, L2, m1, m2, g = sp.symbols('L1 L2 m1 m2 g')
tau1, tau2 = sp.Function('tau1')(t), sp.Function('tau2')(t)

# Derivatives of the angles
theta1_dot = sp.diff(theta1, t)
theta2_dot = sp.diff(theta2, t)
theta1_ddot = sp.diff(theta1_dot, t)
theta2_ddot = sp.diff(theta2_dot, t)

# Position of the center of masses
x1 = L1 * sp.sin(theta1)
y1 = -L1 * sp.cos(theta1)
x2 = x1 + L2 * sp.sin(theta2)
y2 = y1 - L2 * sp.cos(theta2)

# Velocities of the center of masses
vx1 = sp.diff(x1, t)
vy1 = sp.diff(y1, t)
vx2 = sp.diff(x2, t)
vy2 = sp.diff(y2, t)

# Kinetic and potential energy
T1 = 1/2 * m1 * (vx1**2 + vy1**2)
T2 = 1/2 * m2 * (vx2**2 + vy2**2)
V1 = m1 * g * y1
V2 = m2 * g * y2

Lagrangian = (T1 + T2) - (V1 + V2)